How to think about the notebook now

Introduction & Goal
“Build baseline hospitalization model on MEPS 2023 + fairness meta table.”

Load & Prepare Data
(Polars load, rename, convert to Pandas.)

Feature Engineering & Target
(Categorical list, dummies, define hospitalized.)

EDA (optional)
(Class balance, summary stats, correlation with target.)

Baseline Logistic Regression
(Train/test split, model, metrics.)

Fairness Meta Table
(Subset columns, attach predictions for either test set or full dataset.)

In [1]:
import pandas as pd
import polars as pl
import joblib

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    roc_auc_score,
    classification_report,
    precision_recall_curve,
)

# Display options (optional)
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

In [2]:
data_path = "./data/meps_2023/"
main_df = "h251.xlsx"

### Loading and preparing data

In [3]:
# reading as polars for speed
df_pl_raw = pl.read_excel(f"{data_path}{main_df}")

In [4]:
# mapping the columns for clean names
rename_map = {
    "DUPERSID": "person_id",
    "AGELAST": "age",
    "SEX": "sex",
    "RACEV1X": "race_simple",
    "RACETHX": "race_ethnicity",
    "HISPANX": "hispanic",
    "EDUCYR": "education_years",
    "FAMINC23": "family_income",
    "POVCAT23": "poverty_category",
    "INSCOV23": "insurance_coverage",
    "INSURC23": "insurance_category",
    "REGION23": "region",
    "BORNUSA": "born_in_usa",
    "YRSINUS": "years_in_us",
    "RTHLTH53": "self_rated_health",
    "MNHLTH53": "self_rated_mental_health",
    "ADSMOK42": "smoker",
    "HIBPDX": "hypertension_dx",
    "CHDDX": "coronary_hd_dx",
    "ASTHDX": "asthma_dx",
    "DIABDX_M18": "diabetes_dx",
    "OBTOTV23": "office_visits",
    "OPTOTV23": "outpatient_visits",
    "ERTOT23": "er_visits",
    "DVTOT23": "total_visits",
    "TOTEXP23": "total_expenditures",
    "IPDIS23": "inpatient_discharges",
    "IPTEXP23": "inpatient_expenditures",
    "IPNGTD23": "inpatient_nights",
}

In [5]:
# Use the rename_map keys as keep_cols so nothing gets truncated
cols_to_keep = list(rename_map.keys())
df_pl = df_pl_raw.select(cols_to_keep).rename(rename_map)

In [6]:
# convert to pandas df
df_pd = df_pl.to_pandas()

print(df_pd.shape)
print(df_pd.head())

(18919, 29)
    person_id  age  sex  race_simple  race_ethnicity  hispanic  \
0  2790002101   58    2            2               3         2   
1  2790002102   27    1            2               3         2   
2  2790004101   49    2            1               2         2   
3  2790006101   75    2            1               2         2   
4  2790006102   23    1            1               2         2   

   education_years  family_income  poverty_category  insurance_coverage  \
0               17         130700                 5                   1   
1               12         130700                 5                   1   
2               17          87000                 5                   1   
3               12          38000                 4                   2   
4               11          38000                 4                   2   

   insurance_category  region  born_in_usa  years_in_us  self_rated_health  \
0                   1       2            1           -1       

MEPS uses special negative values such as –1 (“inapplicable”), –7 (“refused”), –8 (“don’t know”), and –9 (“not ascertained”) to indicate different types of missing data. These codes are not real measurements, and leaving them in the dataset would mislead both descriptive statistics and the machine learning model—for example, the model might incorrectly learn patterns based on “–8 office visits” or “–9 dollars spent,” which have no meaningful interpretation.

Used chatgpt-5 to help me clean this

In [7]:
# 4. Handle MEPS special codes for numeric columns
#    (-1, -7, -8, -9 → NaN)

SPECIAL_CODES = [-1, -7, -8, -9]

numeric_cols = [
    "age",
    "education_years",
    "family_income",
    "years_in_us",
    "office_visits",
    "outpatient_visits",
    "er_visits",
    "total_visits",
    "total_expenditures",
    "inpatient_discharges",
    "inpatient_expenditures",
    "inpatient_nights",
]

df_pd[numeric_cols] = df_pd[numeric_cols].replace(SPECIAL_CODES, pd.NA)

# Quick sanity check
df_pd[numeric_cols].describe().T

,count,mean,std,min,25%,50%,75%,max
age,18919.0,43.716581,23.939550,0.0,23.0,45.0,64.0,85.0
family_income,18919.0,98815.895555,91575.419638,-230.0,35000.0,72800.0,137484.0,747346.0
office_visits,18919.0,7.140758,13.784344,0.0,0.0,2.0,8.0,419.0
outpatient_visits,18919.0,1.095195,4.489569,0.0,0.0,0.0,1.0,178.0
er_visits,18919.0,0.224166,0.672914,0.0,0.0,0.0,0.0,22.0
total_visits,18919.0,1.035943,1.688402,0.0,0.0,0.0,2.0,25.0
total_expenditures,18919.0,8422.054125,21664.250470,0.0,299.5,1816.0,7087.0,574675.0
inpatient_discharges,18919.0,0.096728,0.394552,0.0,0.0,0.0,0.0,10.0
inpatient_expenditures,18919.0,1830.599397,11540.246170,0.0,0.0,0.0,0.0,458360.0
inpatient_nights,18919.0,0.535599,4.267027,0.0,0.0,0.0,0.0,264.0


In [8]:
# Categorical columns for one-hot encoding
categorical_cols = [
    "sex",
    "race_simple",
    "race_ethnicity",
    "hispanic",
    "poverty_category",
    "insurance_coverage",
    "insurance_category",
    "region",
    "born_in_usa",
    "self_rated_health",
    "self_rated_mental_health",
    "smoker",
    "hypertension_dx",
    "coronary_hd_dx",
    "asthma_dx",
    "diabetes_dx",
]

# One-hot encoding
df_ohe = pd.get_dummies(df_pd, columns=categorical_cols, drop_first=False)

# Defining target
df_ohe["hospitalized"] = (df_ohe["inpatient_expenditures"] > 0).astype(int)

In [9]:
"""
Used Chatgpt-5 on 21 Nov to clean and encode the feature columns
"""

'\nUsed Chatgpt-5 on 21 Nov to clean and encode the feature columns\n'

### Quick Sanity Checks and EDA

In [10]:
display(df_ohe.head())

,person_id,age,education_years,family_income,years_in_us,office_visits,outpatient_visits,er_visits,total_visits,total_expenditures,inpatient_discharges,inpatient_expenditures,inpatient_nights,sex_1,sex_2,race_simple_1,race_simple_2,race_simple_3,race_simple_4,race_simple_6,race_ethnicity_1,race_ethnicity_2,race_ethnicity_3,race_ethnicity_4,race_ethnicity_5,hispanic_1,hispanic_2,poverty_category_1,poverty_category_2,poverty_category_3,poverty_category_4,poverty_category_5,insurance_coverage_1,insurance_coverage_2,insurance_coverage_3,insurance_category_1,insurance_category_2,insurance_category_3,insurance_category_4,insurance_category_5,insurance_category_6,insurance_category_7,insurance_category_8,region_-1,region_1,region_2,region_3,region_4,born_in_usa_-8,born_in_usa_-7,born_in_usa_-1,born_in_usa_1,born_in_usa_2,self_rated_health_-8,self_rated_health_-7,self_rated_health_-1,self_rated_health_1,self_rated_health_2,self_rated_health_3,self_rated_health_4,self_rated_health_5,self_rated_mental_health_-8,self_rated_mental_health_-7,self_rated_mental_health_-1,self_rated_mental_health_1,self_rated_mental_health_2,self_rated_mental_health_3,self_rated_mental_health_4,self_rated_mental_health_5,smoker_-15,smoker_-1,smoker_1,smoker_2,hypertension_dx_-8,hypertension_dx_-7,hypertension_dx_-1,hypertension_dx_1,hypertension_dx_2,coronary_hd_dx_-8,coronary_hd_dx_-7,coronary_hd_dx_-1,coronary_hd_dx_1,coronary_hd_dx_2,asthma_dx_-8,asthma_dx_-7,asthma_dx_-1,asthma_dx_1,asthma_dx_2,diabetes_dx_-8,diabetes_dx_-7,diabetes_dx_-1,diabetes_dx_1,diabetes_dx_2,hospitalized
0,2790002101,58,17,130700,<NA>,3,1,0,0,646,0,0,0,False,True,False,True,False,False,False,False,False,True,False,False,False,True,False,False,False,False,True,True,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False,False,True,False,False,False,False,True,False,False,False,False,True,False,False,False,False,True,False,False,False,True,False,0
1,2790002102,27,12,130700,<NA>,1,0,0,2,1894,0,0,0,True,False,False,True,False,False,False,False,False,True,False,False,False,True,False,False,False,False,True,True,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,True,False,False,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False,True,False,False,False,True,False,False,False,False,False,True,0
2,2790004101,49,17,87000,<NA>,1,0,0,1,986,0,0,0,False,True,True,False,False,False,False,False,True,False,False,False,False,True,False,False,False,False,True,True,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,True,False,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False,False,True,False,False,False,False,True,False,False,False,False,True,False,False,False,False,True,0
3,2790006101,75,12,38000,<NA>,3,0,0,0,1312,0,0,0,False,True,True,False,False,False,False,False,True,False,False,False,False,True,False,False,False,True,False,False,True,False,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False,False,True,False,False,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False,False,False,True,False,False,False,False,True,False,False,False,False,False,True,False,False,False,False,True,False,False,False,True,False,0
4,2790006102,23,11,38000,<NA>,0,0,0,0,0,0,0,0,True,False,True,False,False,False,False,False,True,False,False,False,False,True,False,False,False,True,False,False,True,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,True,False,False,False,False,False,True,False,False,False,False,False,False,False,True,False,Fals

In [11]:
# Checking for class imbalance
print(df_ohe["hospitalized"].value_counts(normalize=True))

hospitalized
0    0.926265
1    0.073735
Name: proportion, dtype: float64


Only 7.37% of individuals in the MEPS dataset experienced any inpatient hospitalization while 93.7% did not, which means the dataset is extremely imbalanced. 

In [12]:
# Basic info
print(df_ohe.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18919 entries, 0 to 18918
Data columns (total 94 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   person_id                    18919 non-null  object
 1   age                          18919 non-null  int64 
 2   education_years              17713 non-null  object
 3   family_income                18919 non-null  int64 
 4   years_in_us                  3080 non-null   object
 5   office_visits                18919 non-null  int64 
 6   outpatient_visits            18919 non-null  int64 
 7   er_visits                    18919 non-null  int64 
 8   total_visits                 18919 non-null  int64 
 9   total_expenditures           18919 non-null  int64 
 10  inpatient_discharges         18919 non-null  int64 
 11  inpatient_expenditures       18919 non-null  int64 
 12  inpatient_nights             18919 non-null  int64 
 13  sex_1                        18

The processed dataset has 18,919 rows and 94 columns, with no missing values across any features. Most features are one-hot encoded categorical features stored as boolean columns, while 13 columns are numeric (age, income, visits, expenditures, etc.), and one identifier column is an object type. The presence of many dummy variables means the model can capture group-specific patterns, but it also increases the risk of multicollinearity and noisy predictors — though logistic regression can generally handle this. Overall, the dataset is clean, well-structured, and ready for supervised learning.

In [13]:
# Summary stats
print(df_ohe.describe().T)

                          count          mean           std    min      25%  \
age                     18919.0     43.716581     23.939550    0.0     23.0   
family_income           18919.0  98815.895555  91575.419638 -230.0  35000.0   
office_visits           18919.0      7.140758     13.784344    0.0      0.0   
outpatient_visits       18919.0      1.095195      4.489569    0.0      0.0   
er_visits               18919.0      0.224166      0.672914    0.0      0.0   
total_visits            18919.0      1.035943      1.688402    0.0      0.0   
total_expenditures      18919.0   8422.054125  21664.250470    0.0    299.5   
inpatient_discharges    18919.0      0.096728      0.394552    0.0      0.0   
inpatient_expenditures  18919.0   1830.599397  11540.246170    0.0      0.0   
inpatient_nights        18919.0      0.535599      4.267027    0.0      0.0   
hospitalized            18919.0      0.073735      0.261347    0.0      0.0   

                            50%       75%       max

In [14]:
# Correlation with target
corr_with_y = df_ohe.corr(numeric_only=True)["hospitalized"].sort_values(
    ascending=False
)

In [15]:
corr_with_y.head(30)

hospitalized                   1.000000
inpatient_discharges           0.865864
inpatient_expenditures         0.562237
total_expenditures             0.451419
inpatient_nights               0.444087
er_visits                      0.416078
age                            0.175374
hypertension_dx_1              0.164760
coronary_hd_dx_1               0.155536
self_rated_health_-1           0.150579
self_rated_mental_health_-1    0.150579
outpatient_visits              0.146469
office_visits                  0.141957
diabetes_dx_1                  0.126824
region_-1                      0.117282
self_rated_health_5            0.115014
insurance_category_4           0.109697
self_rated_health_4            0.107225
insurance_coverage_2           0.106903
insurance_category_6           0.103033
insurance_category_5           0.068628
self_rated_mental_health_4     0.064495
race_ethnicity_2               0.050264
asthma_dx_1                    0.049784
self_rated_mental_health_5     0.048248


### Train/test split & baseline logistic regression

In [16]:
# 9. Train/test split & baseline logistic regression

# Columns to drop from X (ID + explicit inpatient outcomes)
cols_to_drop = [
    "hospitalized",
    "person_id",
    "inpatient_expenditures",
    "inpatient_nights",
    "inpatient_discharges",
]

# Convert extension NA to np.nan
df_ohe = df_ohe.astype("float64", errors="ignore")
# Fill remaining missing values (simple baseline approach)
df_ohe = df_ohe.fillna(0)

# Features and target
X = df_ohe.drop(columns=[c for c in cols_to_drop if c in df_ohe.columns])
y = df_ohe["hospitalized"]


# Train/test split (stratified)
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.25,
    random_state=42,
    stratify=y,
)

# Baseline logistic regression (class_weight balanced for imbalance)
model = LogisticRegression(
    max_iter=1000,
    class_weight="balanced",
    n_jobs=-1,
)

# Fit model
model.fit(X_train, y_train)

# Predictions
y_pred = model.predict(X_test)
y_proba = model.predict_proba(X_test)[:, 1]

/var/folders/f4/vybgzbbx1sg165hn_kn98j0m0000gn/T/ipykernel_18536/3336444422.py:15: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_ohe = df_ohe.fillna(0)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [17]:
"""
Used Chatpgt-5 on 21st Nove to fix this error: TypeError: float() argument must be a string or a real number, not 'NAType'
Got these code lines from Chatgpt-5:

# Convert extension NA to np.nan
df_ohe = df_ohe.astype("float64", errors="ignore")

# Fill remaining missing values (simple baseline approach)
df_ohe = df_ohe.fillna(0)
"""

'\nUsed Chatpgt-5 on 21st Nove to fix this error: TypeError: float() argument must be a string or a real number, not \'NAType\'\nGot these code lines from Chatgpt-5:\n\n# Convert extension NA to np.nan\ndf_ohe = df_ohe.astype("float64", errors="ignore")\n\n# Fill remaining missing values (simple baseline approach)\ndf_ohe = df_ohe.fillna(0)\n'

In [18]:
# Evaluation: ROC-AUC, PR curve, classification report

# Precision–recall curve (dunno if i want this)
prec, rec, thresh = precision_recall_curve(y_test, y_proba)

# ROC-AUC
roc_auc = roc_auc_score(y_test, y_proba)
print("ROC-AUC:", roc_auc)

# Classification report
print(classification_report(y_test, y_pred))

ROC-AUC: 0.9508570808172042
              precision    recall  f1-score   support

         0.0       0.99      0.90      0.94      4381
         1.0       0.41      0.88      0.56       349

    accuracy                           0.90      4730
   macro avg       0.70      0.89      0.75      4730
weighted avg       0.95      0.90      0.91      4730



In [19]:
# 11. Refit model on full dataset (for Fairness Playground)

model_full = LogisticRegression(
    max_iter=1000,
    class_weight="balanced",
    n_jobs=-1,
)

model_full.fit(X, y)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(class_weight='balanced', max_iter=1000, n_jobs=-1)

In [20]:
# 12. Build df_meta for fairness / XAI web app

# Columns from df_pd to carry into the meta table
fairness_cols = [
    "person_id",
    "age",
    "sex",
    "race_ethnicity",
    "hispanic",
    "poverty_category",
    "insurance_coverage",
    "family_income",
    "self_rated_health",
    "self_rated_mental_health",
]

# Start meta table from df_pd
df_meta = df_pd[fairness_cols].copy()

# Add true label from df_ohe
df_meta["hospitalized"] = df_ohe["hospitalized"].values

# Add model predictions
df_meta["pred_prob"] = model_full.predict_proba(X)[:, 1]
df_meta["pred_label"] = model_full.predict(X)

df_meta.head()

,person_id,age,sex,race_ethnicity,hispanic,poverty_category,insurance_coverage,family_income,self_rated_health,self_rated_mental_health,hospitalized,pred_prob,pred_label
0,2790002101,58,2,3,2,5,1,130700,4,3,0.0,0.154102,0.0
1,2790002102,27,1,3,2,5,1,130700,2,2,0.0,0.082755,0.0
2,2790004101,49,2,2,2,5,1,87000,1,1,0.0,0.097175,0.0
3,2790006101,75,2,2,2,4,2,38000,2,2,0.0,0.269318,0.0
4,2790006102,23,1,2,2,4,2,38000,2,2,0.0,0.134672,0.0


In [21]:
df_meta.to_parquet("./data/meps_2023/meps_2023_meta.parquet", index=False)

In [23]:
joblib.dump(model_full, "./data/models/logreg_hospitalization.pkl")

['./data/models/logreg_hospitalization.pkl']